In [ ]:
!pip install -r requirements.txt

In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
data_folder = "./data/"

## Vorverarbeitung

In [19]:

data = pd.read_csv(data_folder + 'ukr-civharm-2023-05-25.csv')  

# Transformiere 'date' Spalte in datetime-Format
data['date'] = pd.to_datetime(data['date'])

# Speichern Sie den aufbereiteten Datensatz als neue CSV-Datei
data.to_csv('processed_data.csv', index=False)


In [20]:
data_folder = "./data/"

In [34]:
import pandas as pd

data = pd.read_csv(data_folder + 'ukr-civharm-2023-05-25.csv')

# Transformiere 'date' Spalte in datetime-Format
# append 12:00:00 to date on every line
data['date'] = pd.to_datetime(data['date'] + ' 12:00:00')

# Speichern Sie den aufbereiteten Datensatz als neue CSV-Datei
data.to_csv('processed_data.csv', index=False)


In [19]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

#Laden Sie Ihr Daten-DataFrame
df = pd.read_csv(data_folder + 'ukr-civharm-2023-05-25.csv')  

# Laden Sie die GeoJSON-Datei
geo_data = gpd.read_file('data/geoBoundaries-UKR-ADM1_simplified.geojson')

# Erstellen Sie Geopandas-DataFrame aus Ihrem ursprünglichen DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Stellen Sie sicher, dass beide Geodataframes denselben Koordinatensystem haben
gdf = gdf.set_crs(geo_data.crs)

# Ordnen Sie jeden Punkt dem entsprechenden Polygon in der GeoJSON-Datei zu
df['region'] = gpd.sjoin(gdf, geo_data, how='left', op='within')['shapeName']

# Transformiere 'date' Spalte in datetime-Format
# append 12:00:00 to date on every line
df['date'] = pd.to_datetime(df['date'] + ' 12:00:00')

# Stellen Sie sicher, dass 'date' ein datetime-Objekt ist
df['date'] = pd.to_datetime(df['date'])

# Sortieren Sie den DataFrame nach 'date' und 'region'
df = df.sort_values(['region', 'date', 'id'])

# Fügen Sie die kumulative Anzahl der Ereignisse pro Region hinzu
df['cumulative_events_per_region'] = df.groupby('region').cumcount() + 1

for i in range(1, 8):
    print(df['cumulative_events_per_region'][i])
    print(df['region'][i])
    print(df['date'][i])

# Berechnen Sie die Zentroide für jede Region in der GeoJSON-Datei
geo_data['centroid'] = geo_data.geometry.centroid

# Extrahieren Sie die Längen- und Breitengrade der Zentroide in separate Spalten
geo_data['centroid_lon'] = geo_data['centroid'].apply(lambda point: point.x)
geo_data['centroid_lat'] = geo_data['centroid'].apply(lambda point: point.y)

# Entfernen Sie die 'centroid'-Spalte, da wir sie nicht mehr brauchen
geo_data = geo_data.drop('centroid', axis=1)

# Führen Sie einen Merge mit dem ursprünglichen DataFrame durch, um die Zentroiden zuzuordnen
df = pd.merge(df, geo_data[['shapeName', 'centroid_lon', 'centroid_lat']], 
              left_on='region', right_on='shapeName', how='left')

# Entfernen Sie die 'shapeName'-Spalte, da wir sie nicht mehr brauchen
df = df.drop('shapeName', axis=1)


print(df.head())
df.to_csv('processed_data_time_region.csv', index=False)

1.0
Cherkasy Oblast
2022-02-24 12:00:00
1.0
Kyiv
2022-02-24 12:00:00
1.0
Donetsk Oblast
2022-02-24 12:00:00
1.0
Kherson Oblast
2022-02-24 12:00:00
2.0
Donetsk Oblast
2022-02-24 12:00:00
3.0
Donetsk Oblast
2022-02-24 12:00:00
4.0
Donetsk Oblast
2022-02-24 12:00:00
        id                date   latitude  longitude          location  \
0  CIV0002 2022-02-24 12:00:00  48.748564  30.218515              Uman   
1  CIV1122 2022-06-26 12:00:00  49.474788  32.033788          Cherkasy   
2  CIV1600 2022-10-31 12:00:00  49.763071  31.470421  Cherkassy region   
3  CIV2023 2023-04-28 12:00:00  48.766328  30.217014               NaN   
4  CIV0042 2022-02-26 12:00:00  51.524277  31.325445         Chernihiv   

                                         description  \
0  Civilians hit by what appears to have been art...   
1   Explosion in the immediate vicinity of a bridge.   
2      A power plant reported as hit near Cherkassy.   
3  [Graphic: covered dead bodies]20 people report...   
4  Apartmen

/opt/homebrew/anaconda3/envs/projekt_vis/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning:

The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.

/var/folders/27/wbxl2gm97fx87d4kg4dnfznw0000gn/T/ipykernel_41278/2519674894.py:39: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [11]:
for i in range(1, 10):
    print(df["region"][i])
    print(df["cumulative_events_per_region"][i])

Cherkasy Oblast
2.0
Cherkasy Oblast
3.0
Cherkasy Oblast
4.0
Chernihiv Oblast
1.0
Chernihiv Oblast
2.0
Chernihiv Oblast
3.0
Chernihiv Oblast
4.0
Chernihiv Oblast
5.0
Chernihiv Oblast
6.0


In [4]:
import plotly.express as px
import plotly.io as pio

#Laden Sie Ihr Daten-DataFrame
df = pd.read_csv("processed_data_time_region.csv")  

# Zählen Sie die Anzahl der Ereignisse in jeder Region
region_counts = df['region'].value_counts().reset_index()
region_counts.columns = ['region', 'count']

# Erstellen Sie das Balkendiagramm
fig = px.bar(region_counts, x='region', y='count', title='Anzahl der Ereignisse pro Region',
             labels={'count':'Anzahl der Ereignisse', 'region':'Region'})

# Diagramm anzeigen
fig.show()
pio.write_html(fig, file="region_count.html", auto_open=True)